In [ ]:
!mv kaggle.json /root/.kaggle/

In [ ]:
!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus

 64% 17.0M/26.4M [00:00<00:00, 28.9MB/s]
100% 26.4M/26.4M [00:00<00:00, 45.0MB/s]


In [ ]:
!unzip '/content/entity-annotated-corpus.zip'

Archive:  /content/entity-annotated-corpus.zip
  inflating: ner.csv                 
  inflating: ner_dataset.csv         


In [122]:
import pandas as pd
import numpy as np
import re
import nltk
import random

In [6]:
dataset=pd.read_csv('/content/ner_dataset.csv',encoding='cp1252', error_bad_lines=False)

In [7]:
dataset.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [8]:
' '.join(dataset.iloc[0:5,1].values)

'Thousands of demonstrators have marched'

In [9]:
data=dataset.iloc[:,:3].values

In [10]:
data[1:40]

array([[nan, 'of', 'IN'],
       [nan, 'demonstrators', 'NNS'],
       [nan, 'have', 'VBP'],
       [nan, 'marched', 'VBN'],
       [nan, 'through', 'IN'],
       [nan, 'London', 'NNP'],
       [nan, 'to', 'TO'],
       [nan, 'protest', 'VB'],
       [nan, 'the', 'DT'],
       [nan, 'war', 'NN'],
       [nan, 'in', 'IN'],
       [nan, 'Iraq', 'NNP'],
       [nan, 'and', 'CC'],
       [nan, 'demand', 'VB'],
       [nan, 'the', 'DT'],
       [nan, 'withdrawal', 'NN'],
       [nan, 'of', 'IN'],
       [nan, 'British', 'JJ'],
       [nan, 'troops', 'NNS'],
       [nan, 'from', 'IN'],
       [nan, 'that', 'DT'],
       [nan, 'country', 'NN'],
       [nan, '.', '.'],
       ['Sentence: 2', 'Families', 'NNS'],
       [nan, 'of', 'IN'],
       [nan, 'soldiers', 'NNS'],
       [nan, 'killed', 'VBN'],
       [nan, 'in', 'IN'],
       [nan, 'the', 'DT'],
       [nan, 'conflict', 'NN'],
       [nan, 'joined', 'VBD'],
       [nan, 'the', 'DT'],
       [nan, 'protesters', 'NNS'],
       [nan, 'who',

In [11]:
current=0
for i in range(data.shape[0]):
  if(type(data[i,0])==str):
    current=current+1
  data[i,0]=current

In [288]:
vocab=dict([ (v,k) for (k,v) in list(enumerate(set([x.lower() for x in list(set(dataset.iloc[:,1].values))])))] )
tags=dict([ (v,k) for (k,v) in enumerate(list(set(dataset.iloc[:,2].values))) ] )

tag_reverse=dict([(v,k) for (k,v) in list(tags.items()) ])
vocab_reverse=dict([(v,k) for (k,v) in list(vocab.items()) ])

In [17]:
len(tags)

42

In [18]:
Transitions=np.zeros((len(tags)+1,len(tags)))

In [21]:
#Populate Transitions

for i in range(1,data.shape[0]-1):
  first=tags[data[i,2]]
  second=tags[data[i+1,2]]

  Transitions[first,second]+=1

In [22]:
for i in range(data.shape[0]-1):
  if data[i,0]-data[i+1,0]!=0 :
    Transitions[0,tags[data[i+1,2]]]+=1


In [23]:
Transitions+=1e-5

In [24]:
for i in range(0,Transitions.shape[0]):
  sum=0
  for j in range(0,Transitions.shape[1]):
    sum+=Transitions[i,j]
  Transitions[i]=Transitions[i]/sum

In [25]:
Transitions.shape

(43, 42)

In [26]:
Emissions=np.zeros((len(tags),len(vocab)))

In [27]:
for i in range(data.shape[0]):
  Emissions[tags[data[i,2]],vocab[data[i,1].lower()]]+=1

In [28]:
Emissions+=1e-5
for i in range(Emissions.shape[0]):
  sum=0
  for j in range(Emissions.shape[1]):
    sum+=Emissions[i,j]
  Emissions[i]=Emissions[i]/sum

In [289]:
def predict_POS(statement):
  statement=statement.lower()
  words=(nltk.tokenize.TweetTokenizer()).tokenize(statement)

  # Viterbi Inits
  C=np.zeros((len(tags),len(words)))
  D=np.zeros((len(tags),len(words)),dtype=int)

  # Assign Initials..
  if words[0] in vocab :
    C[:,0]=Transitions[0,:]*Emissions[:,vocab[words[0]]]
  else :
    C[:,0]=Transitions[0,:]*1.0
  D[:,0]=-1

  # Populating C and D 

  for i in range(0,len(tags)):
    for j in range(1,len(words)):
      try:
        temp=[(C[k,j-1])*Transitions[k+1,i]*Emissions[i,vocab[words[j]]] for k in range(len(tags)) ]
        C[i,j]=np.max(temp)
        
        D[i,j]=np.argmax(temp)
      except :
        temp=[(C[k,j-1])*Transitions[k+1,i]*1.0 for k in range(len(tags)) ]
        C[i,j]=np.max(temp)
        
        D[i,j]=np.argmax(temp)

  # BackTrack 
  result=[]
  temp=list(C[:,C.shape[1]-1]).index(random.choices(C[:,C.shape[1]-1],C[:,C.shape[1]-1]/np.sum(C[:,C.shape[1]-1])))
  result.append(temp)
  for i in range(D.shape[1]-1,0,-1):
    result.append(D[temp,i])
    temp=list(C[:,i-1]).index(random.choices(C[:,i-1],(C[:,i-1]/np.sum(C[:,i-1])) ))

  result.reverse()

  
  for i in range(len(result)):
    result[i]=tag_reverse[result[i]]

  return result


In [290]:
predict_POS('my name is gajesh ladhar')

['PRP$', 'NN', 'VBZ', 'VBG', 'VBG']

In [320]:
#Questions Answers

Q='what you ate 2 months ago'
corpus='i eat banana 2 months ago'
c_list=(nltk.tokenize.TweetTokenizer()).tokenize(corpus)
q_list=(nltk.tokenize.TweetTokenizer()).tokenize(Q)
nouns=[]
for i,word in enumerate(predict_POS(corpus)):
  if word=="NN":
    nouns.append(c_list[i])
for word in nouns:
  if word in q_list:
    nouns.remove(word)
print(nouns)

['banana']


In [296]:
# Text Generation with Proper Grammer

state=list(Transitions[0,:]).index(random.choices(Transitions[0,:],Transitions[0,:]))
word=vocab_reverse[list(Emissions[state,:]).index(random.choices(Emissions[state,:],Emissions[state,:]))]
print(word,end=' ')

for i in range(40):
  state=list(Transitions[state+1,:]).index(random.choices(Transitions[state+1,:],Transitions[state+1,:]))
  word=vocab_reverse[list(Emissions[state,:]).index(random.choices(Emissions[state,:],Emissions[state,:]))]
  print(word,end=' ')


it termed said wish its friday voting face the experiments one forced is temporarily health regain the of violence stop soft identify her after korean 15 killed says weeks speech not sharing announce natural a out british to has operation travel 

Testing with NLTK's Tagger


In [209]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [307]:
nltk.pos_tag((nltk.tokenize.TweetTokenizer()).tokenize('he look very good but gajesh'))

[('he', 'PRP'),
 ('look', 'VB'),
 ('very', 'RB'),
 ('good', 'JJ'),
 ('but', 'CC'),
 ('gajesh', 'JJ')]

In [306]:
predict_POS('he look very good but gajesh')

['PRP', 'VB', 'JJ', 'JJ', 'CC', 'VBG']